In [1]:
import pandas as pd
import datetime
import os
import re

import assets.helper as b3
import assets.functions as run

import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px


## Load Files

In [2]:
folder = r'C:\Users\faust\Downloads'


In [3]:
filename = 'r_pac_atend (2).csv_cleaned (2).csv'
df_mv = pd.read_csv(os.path.join(folder, filename))
df_mv['MES'] = pd.to_datetime(df_mv['MES'])
df_mv['DIA'] = pd.to_datetime(df_mv['DIA'])


In [4]:
filename = 'GERCON 12 meses.docx_cleaned.csv'
df_gercon = pd.read_csv(os.path.join(folder, filename))
df_gercon['MES'] = pd.to_datetime(df_gercon['MES'])
df_gercon['DIA'] = pd.to_datetime(df_gercon['DIA'])


In [5]:
filename = 'recursos opticos planilha - Página1.csv_cleaned.csv'
df_recursos_opticos = pd.read_csv(os.path.join(folder, filename))
df_recursos_opticos['TESTE'] = pd.to_datetime(df_recursos_opticos['TESTE'])
df_recursos_opticos['PEDIDO'] = pd.to_datetime(df_recursos_opticos['PEDIDO'])
df_recursos_opticos['VALOR'] = df_recursos_opticos['VALOR'].str.replace('R$', '', regex=False).str.replace(',', '.', regex=False).str.strip()
df_recursos_opticos['VALOR'] = df_recursos_opticos['VALOR'].replace(992.92, 992.62)
df_recursos_opticos['VALOR'] = pd.to_numeric(df_recursos_opticos['VALOR'], errors='coerce')
df_recursos_opticos.dropna(subset=['VALOR'], inplace=True)


In [6]:
filename = 'Cópia de PLANILHA ENTREGA DE BENGALAS OUT 2022- OUT  2023 - Prótese - Jan_fev_2022.csv_cleaned.csv'
df_bengala = pd.read_csv(os.path.join(folder, filename))
df_bengala['Data da entrega'] = pd.to_datetime(df_bengala['Data da entrega'])


In [7]:
filename = 'Valores materiais CRV.xlsx - Planilha1 (1).csv_cleaned.csv'
df_sigtap = pd.read_csv(os.path.join(folder, filename))
df_sigtap['FORNECEDOR'] = pd.to_numeric(df_sigtap['FORNECEDOR'], errors='coerce')
df_sigtap['SIGTAP'] = pd.to_numeric(df_sigtap['SIGTAP'], errors='coerce')
new_rows = [
    {'OPME': 'CONSULTA MÉDICA', 'FORNECEDOR': 0.00, 'SIGTAP': 10.00},
    {'OPME': 'CONSULTA MULTIDISCIPLINAR', 'FORNECEDOR': 0.00, 'SIGTAP': 12.00}
]
df_sigtap = pd.concat([df_sigtap, pd.DataFrame(new_rows)], ignore_index=True)



In [8]:
filename = 'LISTA DE PREÇOS ATUAL 2023 - BAIXA VISÃO - SOMENTE REVENDA (2) (2).pdf_cleaned.csv'
df_provista = pd.read_csv(os.path.join(folder, filename))
df_provista['VALOR'] = df_provista['VALOR'].str.replace('R$', '', regex=False).str.replace(',', '.', regex=False).str.strip()
df_provista['VALOR'] = pd.to_numeric(df_provista['VALOR'], errors='coerce')


## Preliminar Adjustments and Alignments

In [9]:
medico_mapping = {
    'GRAZIELA QUARESMA DA F': 'ASSISTENCIA SOCIAL',
    'VERONICA MATTOS RODRI': 'ORIENTAÇÃO E MOBILIDADE',
    'FERNANDA DOS SANTOS SI': 'ORIENTAÇÃO E MOBILIDADE',
    'TIAGO FRANCO MARTINS': 'PRÓTESE',
    'CAROLINE AKEMI SUE': 'PRÓTESE',
    'VITOR HUGO CAMARGO': 'PRÓTESE',
    'LUIZA BIRCK KLEIN': 'PRÓTESE',
    'CAMILLA ALBUQUERQUE N': 'PRÓTESE',
    'KAROLINY KRAUSE MONIC': 'PRÓTESE',
    'AMANDA SORATTO CITADIN': 'PRÓTESE',
    'SILVIA REGINA BITENCOUR': 'PSICOLOGIA',
    'ALINE FONSECA AFRAUSIN': 'PSICOLOGIA',
    'MARINA GONÇALVES POZZ': 'PSICOLOGIA',
    'ANA MARIA DALL AGNESE': 'PSICOLOGIA',
    'OZELIA FATIMA VASSOLER': 'PSICOLOGIA',
    'RAQUEL FRANZON': 'PSICOLOGIA',
    'GABRIELA DUTRA SILVA': 'RECURSOS ÓPTICOS'
}
df_mv['SETOR'] = df_mv['MEDICO'].map(medico_mapping).fillna('MEDICO')

# Filter df_sigtap for the relevant rows
sigtap_mapping = df_sigtap[df_sigtap['OPME'].isin(['CONSULTA MÉDICA', 'CONSULTA MULTIDISCIPLINAR'])]

# Create a dictionary for mapping
sigtap_valores = sigtap_mapping.set_index('OPME')['SIGTAP'].to_dict()

# Add a new column to df_mv for 'VALOR' based on the 'SETOR' mapping to the sigtap values
df_mv['VALOR'] = df_mv['SETOR'].map(lambda x: sigtap_valores['CONSULTA MÉDICA'] if x == 'MEDICO' else sigtap_valores['CONSULTA MULTIDISCIPLINAR'])


In [10]:
def get_stats(data_series, window=4):
    """
    Calculate moving average, historical average, std+2, and std-2 for a given data series.
    :param data_series: Pandas Series with datetime index
    :param window: The period over which to calculate the moving average
    :return: Dictionary with data and visualization properties
    """
    # Calculating the moving average
    moving_avg = data_series.rolling(window=window).mean()

    # Calculating the historical average
    historical_avg = pd.Series(data_series.mean(), index=data_series.index)

    # Calculating the moving standard deviation
    moving_std = data_series.rolling(window=window).std()

    # Calculating std+2 and std-2 bounds
    std_plus_2 = moving_avg + 2 * moving_std
    std_minus_2 = moving_avg - 2 * moving_std

    return {
        f'{data_series.name}': {'data': data_series, 'mode': 'lines', 'color': 'blue'},
        f'Média Histórica': {'data': historical_avg, 'mode': 'lines', 'color': 'gray'},
        f'Média Móvel {window} períodos': {'data': moving_avg, 'mode': 'lines', 'color': 'red'},
        f'+2 Desvios Padrão': {'data': std_plus_2, 'mode': 'lines', 'color': 'gray'},
        f'-2 Desvios Padrão': {'data': std_minus_2, 'mode': 'lines', 'color': 'gray'}
    }

def round_5(number):
    """
    Rounds a number to the nearest multiple of 5.
    """
    return round(number / 5) * 5


## Análise Histórica da Produção

In [11]:
# Calculating totals
total_atendimentos = df_mv['ATENDIMENTO_CODIGO'].nunique()
total_pacientes = df_mv['PACIENTE_CODIGO'].nunique()
total_medicos = df_mv['MEDICO_CODIGO'].nunique()
total_setores = df_mv['SETOR'].nunique()

# Calculating averages
avg_atendimentos_paciente = total_atendimentos / total_pacientes
avg_atendimentos_setor = round(total_atendimentos / total_setores)
avg_pacientes_setor = round(total_pacientes / total_setores)
avg_setores_por_paciente = df_mv.groupby('PACIENTE_CODIGO')['SETOR'].nunique().mean()

total_atendimentos_por_setor = df_mv.groupby('SETOR')['ATENDIMENTO_CODIGO'].nunique()
total_pacientes_por_setor = df_mv.groupby('SETOR')['PACIENTE_CODIGO'].nunique()
media_atendimentos_por_paciente_setor = total_atendimentos_por_setor / total_pacientes_por_setor

sectors_per_patient = df_mv.groupby('PACIENTE_CODIGO')['SETOR'].nunique()

# Valor total acumulado
total_valor_acumulado = df_mv['VALOR'].sum()

# Valor médio por paciente
avg_valor_por_paciente = df_mv.groupby('PACIENTE_CODIGO')['VALOR'].mean().mean()

# Valor médio por setor
avg_valor_por_setor = df_mv.groupby('SETOR')['VALOR'].mean().mean()

# Valor médio por mês
avg_valor_por_mes = df_mv.resample('M', on='DIA')['VALOR'].mean().mean()



### Análise Numérica

In [12]:
fig = make_subplots(
    rows=1, cols=4,
    specs=[[{"type": "domain"}, {"type": "domain"}, {"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Total Atendimentos", "Total Pacientes", "Total Médicos", "Total Setores")
)

# Adicionando os indicadores
fig.add_trace(go.Indicator(mode="number", value=total_atendimentos), row=1, col=1)
fig.add_trace(go.Indicator(mode="number", value=total_pacientes), row=1, col=2)
fig.add_trace(go.Indicator(mode="number", value=total_medicos), row=1, col=3)
fig.add_trace(go.Indicator(mode="number", value=total_setores), row=1, col=4)

# Atualizando o layout
fig.update_layout(height=300, template='plotly_white')

# Exibir o gráfico
fig.show()


In [13]:
# Updating the plot to include the new average calculation
fig_avg_updated = make_subplots(
    rows=1, cols=4,
    specs=[[{"type": "domain"}, {"type": "domain"}, {"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Média de Atendimentos por Paciente", "Média de Atendimentos por Setor", 
                    "Média de Pacientes por Setor", "Média de Setores por Paciente")
)

# Adding the indicators for averages
fig_avg_updated.add_trace(go.Indicator(mode="number", value=avg_atendimentos_setor), row=1, col=1)
fig_avg_updated.add_trace(go.Indicator(mode="number", value=avg_pacientes_setor), row=1, col=2)
fig_avg_updated.add_trace(go.Indicator(mode="number", value=avg_atendimentos_paciente), row=1, col=3)
fig_avg_updated.add_trace(go.Indicator(mode="number", value=avg_setores_por_paciente), row=1, col=4)

# Updating the layout
fig_avg_updated.update_layout(height=300, template='plotly_white')

# Display the plot
fig_avg_updated.show()


In [14]:
# Adding another column for pie plots in the existing subplot structure
fig_setores_bar_pie = make_subplots(
    rows=3, cols=2,
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "pie"}]],
    subplot_titles=("Total de Atendimentos por Setor", "Distribuição de Atendimentos",
                    "Total de Pacientes por Setor", "Distribuição de Pacientes",
                    "Média de Atendimentos por Paciente por Setor", ""),
    vertical_spacing=0.15
)

# Adding bar plots for total appointments by sector
fig_setores_bar_pie.add_trace(go.Bar(
    x=total_atendimentos_por_setor.index, 
    y=total_atendimentos_por_setor.values,
    name="Total Atendimentos"), 
    row=1, col=1)

# Adding pie plot for distribution of appointments by sector
fig_setores_bar_pie.add_trace(go.Pie(
    labels=total_atendimentos_por_setor.index,
    values=total_atendimentos_por_setor.values,
    name="Distribuição Atendimentos"), 
    row=1, col=2)

# Adding bar plots for total patients by sector
fig_setores_bar_pie.add_trace(go.Bar(
    x=total_pacientes_por_setor.index, 
    y=total_pacientes_por_setor.values,
    name="Total Pacientes"), 
    row=2, col=1)

# Adding pie plot for distribution of patients by sector
fig_setores_bar_pie.add_trace(go.Pie(
    labels=total_pacientes_por_setor.index,
    values=total_pacientes_por_setor.values,
    name="Distribuição Pacientes"), 
    row=2, col=2)

# Adding bar plots for average appointments per patient by sector
fig_setores_bar_pie.add_trace(go.Bar(
    x=media_atendimentos_por_paciente_setor.index, 
    y=media_atendimentos_por_paciente_setor.values,
    name="Média Atendimentos por Paciente"), 
    row=3, col=1)

# Updating the layout
fig_setores_bar_pie.update_layout(
    height=1200, 
    showlegend=True, 
    template='plotly_white'
)

# Display the plot
fig_setores_bar_pie.show()


In [15]:
# Contagem do número de consultas por paciente
appointments_per_patient = df_mv['PACIENTE'].value_counts()

# Criando o histograma com Plotly
fig = go.Figure(go.Histogram(
    x=appointments_per_patient, 
    nbinsx=int(appointments_per_patient.max()),  # Número de bins
    marker_color='skyblue'
))

# Atualizando o layout
fig.update_layout(
    title='Distribuição do Número de Atendimentos por Paciente',
    xaxis_title='Número de Consultas',
    yaxis_title='Número de Pacientes',
    template='plotly_white'
)

# Exibir o gráfico
fig.show()


In [16]:
# Encontrar os setores únicos
unique_sectors = df_mv['SETOR'].unique()

# Determinar o número de linhas para os subplots
n_rows = int(len(unique_sectors) ** 0.5) + 1

# Criando o layout com subplots
fig = make_subplots(rows=n_rows, cols=n_rows, subplot_titles=unique_sectors)

# Definir o número de bins desejado para os histogramas
desired_bins = 25

# Definir o número mínimo de consultas
min_consultas = 3

# Adicionando um histograma para cada setor
for i, sector in enumerate(unique_sectors, start=1):
    # Filtrando o dataframe para o setor atual e contando as consultas por paciente
    sector_data = df_mv[df_mv['SETOR'] == sector]['PACIENTE_CODIGO'].value_counts()

    # Filtrando pacientes com mais de 'min_consultas' consultas
    filtered_sector_data = sector_data[sector_data >= min_consultas]

    # Adicionando o histograma ao subplot correspondente
    fig.add_trace(go.Histogram(x=filtered_sector_data, name=sector, nbinsx=int(filtered_sector_data.max())), row=(i - 1) // n_rows + 1, col=(i - 1) % n_rows + 1)

# Atualizando o layout
fig.update_layout(
    title_text=f'Distribuição do Número de Consultas por Paciente ({min_consultas} ou mais Consultas no Setor)',
    height=900,
    width=1200,
    showlegend=False
)

# Exibir o gráfico
fig.show()


In [17]:
# Encontrar os setores únicos
unique_sectors = df_mv['SETOR'].unique()

# Criando o histograma com Plotly
fig = go.Figure()

# Número mínimo de consultas para incluir no histograma
min_consultas = 2

# Adicionando um histograma para cada setor
for sector in unique_sectors:
    # Filtrando o dataframe para o setor atual e contando as consultas por paciente
    sector_data = df_mv[df_mv['SETOR'] == sector]['PACIENTE_CODIGO'].value_counts()

    # Filtrando para incluir apenas pacientes com consultas >= min_consultas
    filtered_sector_data = sector_data[sector_data >= min_consultas]

    # Adicionando o histograma ao gráfico
    fig.add_trace(go.Histogram(
        x=filtered_sector_data, 
        name=sector, 
        opacity=0.3  # Ajustando a opacidade para visualizar sobreposições
    ))

# Atualizando o layout
fig.update_layout(
    title_text=f'Histograma de Consultas por Paciente ({min_consultas} ou mais Consultas) por Setor',
    xaxis_title='Número de Consultas',
    yaxis_title='Contagem de Pacientes',
    barmode='overlay',  # Modo de sobreposição
    template='plotly_white'
)

# Exibir o gráfico
fig.show()


### Linha do Tempo

In [18]:
# Resampling the data on a weekly basis to count the number of appointments each week
weekly_appointments = df_mv.resample('M', on='DIA').size()
weekly_appointments.name = 'Consultas por Mês'

data = get_stats(weekly_appointments, window=12)

# Criando o gráfico Plotly
fig = go.Figure()

# Adicionando as linhas usando um loop
for name, info in data.items():
    fig.add_trace(go.Scatter(
        x=info['data'].index, 
        y=info['data'], 
        mode=info['mode'], 
        name=name,
        line=dict(color=info['color'])
    ))

# Atualizando o layout
fig.update_layout(
    title=f'{weekly_appointments.name}',
    xaxis_title='Data',
    yaxis_title='Nímero de Consultas',
    legend_title='Legenda',
    template='plotly_white'
)

# Exibir o gráfico
fig.show()


In [19]:
# Resampling the data on a weekly basis to count the number of appointments each week
weekly_appointments = df_mv.resample('W', on='DIA').size()
weekly_appointments.name = 'Consultas por Semana'

data = get_stats(weekly_appointments, window=12)

# Criando o gráfico Plotly
fig = go.Figure()

# Adicionando as linhas usando um loop
for name, info in data.items():
    fig.add_trace(go.Scatter(
        x=info['data'].index, 
        y=info['data'], 
        mode=info['mode'], 
        name=name,
        line=dict(color=info['color'])
    ))

# Atualizando o layout
fig.update_layout(
    title=f'{weekly_appointments.name}',
    xaxis_title='Data',
    yaxis_title='Nímero de Consultas',
    legend_title='Legenda',
    template='plotly_white'
)

# Exibir o gráfico
fig.show()


In [20]:
# Resampling the data on a weekly basis to count the number of appointments each week by sector
weekly_appointments_by_sector = df_mv.groupby('SETOR').resample('W', on='DIA').size().unstack(0).fillna(0)

# Calculating the moving average for each sector
moving_averages_by_sector = weekly_appointments_by_sector.rolling(window=12).mean()

# Creating a new Plotly figure for moving averages by sector
fig_moving_avg = go.Figure()

# Adding lines for each sector's moving average
for sector in moving_averages_by_sector.columns:
    fig_moving_avg.add_trace(go.Scatter(
        x=moving_averages_by_sector.index, 
        y=moving_averages_by_sector[sector], 
        mode='lines', 
        name=sector
    ))

# Updating the layout
fig_moving_avg.update_layout(
    title='Média Móvel Semanal por Setor',
    xaxis_title='Data',
    yaxis_title='Número de Consultas',
    legend_title='Setor',
    template='plotly_white'
)

# Display the plot
fig_moving_avg.show()


### Produção

In [21]:
# Correcting the calculations for average value per sector and per month

# Valor médio por setor (Average Value per Sector)
# This should remain the same as it correctly calculates the mean of the means for each sector
avg_valor_por_setor = df_mv.groupby('SETOR')['VALOR'].mean().mean()
total_valor_por_setor = df_mv.groupby('SETOR')['VALOR'].sum()
avg_total_valor_por_setor = total_valor_por_setor.mean()

monthly_total_valor_por_setor = df_mv.groupby(['SETOR', pd.Grouper(key='DIA', freq='M')])['VALOR'].sum()
avg_monthly_total_valor_por_setor = monthly_total_valor_por_setor.groupby('SETOR').mean()
overall_avg_monthly_total_valor = avg_monthly_total_valor_por_setor.mean()

# Valor total médio por mês (Total Average Value per Month)
# This should calculate the total value for each month and then find the mean of these totals
total_valor_por_mes = df_mv.resample('M', on='DIA')['VALOR'].sum()
avg_total_valor_por_mes = total_valor_por_mes.mean()

# Creating a subplot with indicators for corrected values
fig_indicators_corrected = make_subplots(
    rows=1, cols=4,
    specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]],
    subplot_titles=("Produção Total Acumulada", "Produção Mês", "Produção por Setor", "Produção por Paciente")
)

# Adding the indicators
fig_indicators_corrected.add_trace(go.Indicator(
    mode="number",
    value=total_valor_acumulado), row=1, col=1)

fig_indicators_corrected.add_trace(go.Indicator(
    mode="number",
    value=avg_total_valor_por_mes), row=1, col=2)

fig_indicators_corrected.add_trace(go.Indicator(
    mode="number",
    value=avg_total_valor_por_mes/6), row=1, col=3)

fig_indicators_corrected.add_trace(go.Indicator(
    mode="number",
    value=avg_valor_por_paciente), row=1, col=4)

# Updating the layout
fig_indicators_corrected.update_layout(height=300, template='plotly_white')

# Displaying the subplot
fig_indicators_corrected.show()


In [22]:
import plotly.express as px

# Creating a bar plot to show the average monthly value per sector
fig_bar_avg_monthly_valor = px.bar(
    avg_monthly_total_valor_por_setor.reset_index(),
    x='SETOR',
    y='VALOR',
    title='Produção Mensal por Setor',
    labels={'SETOR': 'Setor', 'VALOR': 'Produção Média Mensal'}
)

# Display the bar plot
fig_bar_avg_monthly_valor.show()


In [23]:
# Resampling the data on a monthly basis to get the total 'VALOR'
total_monthly_valor = df_mv.resample('M', on='DIA')['VALOR'].sum()
total_monthly_valor.name = 'Produção Total Mensal'

# Getting the statistics for the total monthly production
data = get_stats(total_monthly_valor, window=12)

# Creating a Plotly figure for the timeline with statistics
fig_timeline = go.Figure()

# Adding lines using a loop
for name, info in data.items():
    fig_timeline.add_trace(go.Scatter(
        x=info['data'].index, 
        y=info['data'], 
        mode=info['mode'], 
        name=name,
        line=dict(color=info['color'])
    ))

# Updating the layout
fig_timeline.update_layout(
    title='Timeline da Produção Total por Mês',
    xaxis_title='Data',
    yaxis_title='Valor Total',
    legend_title='Indicadores',
    template='plotly_white'
)

# Display the timeline
fig_timeline.show()


In [24]:
# Grouping by month and sector to sum the 'VALOR' for each sector and month
monthly_valor_por_setor = df_mv.groupby(['DIA', 'SETOR'])['VALOR'].sum().unstack('SETOR').fillna(0)

# Resampling the data on a monthly basis to get the total 'VALOR' for each sector
monthly_valor_por_setor = monthly_valor_por_setor.resample('M').sum()

# Creating a stacked bar plot to show the total monthly production by sector over time
fig_stacked_bar = px.bar(
    monthly_valor_por_setor,
    x=monthly_valor_por_setor.index,
    y=monthly_valor_por_setor.columns,
    title='Produção por Setor',
    labels={'value': 'Valor Total', 'variable': 'Setor', 'DIA': 'Data'},
    template='plotly_white'
)

# Update layout to stack bars
fig_stacked_bar.update_layout(barmode='stack')

# Display the stacked bar plot
fig_stacked_bar.show()


## Gercon

In [43]:
df_producao = pd.merge(df_gercon, df_mv, on='PACIENTE', how='outer', suffixes=('_GERCON', '_MV'))


In [44]:
df_producao = pd.merge(df_producao, df_recursos_opticos, on='PACIENTE', how='outer', suffixes=('', '_RECURSO'))


In [45]:
df_producao = pd.merge(df_producao, df_bengala, on='PACIENTE', how='outer', suffixes=('', '_BENGALA'))

In [ ]:
df_producao = pd.merge(df_producao, df_sigtap, on='PACIENTE', how='outer', suffixes=('', '_BENGALA'))

In [79]:
# Filtering the df_producao dataframe for records where 'GERCON' is 'LIVRE' (unoccupied slots)
livre_slots = df_producao[df_producao['GERCON'] == 'LIVRE']

# Calculating the total number of slots and the number of unoccupied (livre) slots
total_slots = len(df_producao)
unoccupied_slots = len(livre_slots)
# Resampling the data on a monthly basis to calculate the total number of slots and unoccupied slots per month
monthly_total_slots = df_producao.resample('M', on='DIA_GERCON').size()
monthly_unoccupied_slots = livre_slots.resample('M', on='DIA_GERCON').size()

# Calculating the average number of unoccupied slots per month
avg_unoccupied_slots_per_month = monthly_unoccupied_slots.mean()

# Displaying the total monthly slots and average monthly unoccupied slots
monthly_total_slots, avg_unoccupied_slots_per_month


(DIA_GERCON
 2022-11-30     87
 2022-12-31     57
 2023-01-31    115
 2023-02-28     61
 2023-03-31     67
 2023-04-30     48
 2023-05-31    102
 2023-06-30     66
 2023-07-31     54
 2023-08-31     50
 2023-09-30     40
 2023-10-31     73
 dtype: int64,
 15.5)

In [80]:
# Adjusting the calculation for vacancy rate
vacancy_rate = 100* unoccupied_slots / total_slots

# Creating a subplot with indicators for the three values: total slots, unoccupied slots, and vacancy rate
fig_indicators_slots = make_subplots(
    rows=1, cols=3,
    specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]],
    subplot_titles=("Total de Horários", "Horários Não Ocupados", "Taxa de Vacância")
)

# Adding indicators for total slots, unoccupied slots, and vacancy rate
fig_indicators_slots.add_trace(go.Indicator(
    mode="number",
    value=monthly_total_slots,), row=1, col=1)

fig_indicators_slots.add_trace(go.Indicator(
    mode="number",
    value=monthly_unoccupied_slots,), row=1, col=2)

fig_indicators_slots.add_trace(go.Indicator(
    mode="number",
    value=vacancy_rate,
    number={'suffix': '%', 'valueformat': '.2f'}), row=1, col=3)

# Updating the layout
fig_indicators_slots.update_layout(height=300, template='plotly_white')

# Displaying the subplot
fig_indicators_slots.show()


ValueError: 
    Invalid value of type 'pandas.core.series.Series' received for the 'value' property of indicator
        Received value: DIA_GERCON
2022-11-30     87
2022-12-31     57
2023-01-31    115
2023-02-28     61
2023-03-31     67
2023-04-30     48
2023-05-31    102
2023-06-30     66
2023-07-31     54
2023-08-31     50
2023-09-30     40
2023-10-31     73
dtype: int64

    The 'value' property is a number and may be specified as:
      - An int or float

In [59]:
paciente_counts = df_producao['PACIENTE'].value_counts()
one_time_patients = paciente_counts[paciente_counts == 1].index

# Filtering df_producao for these unique patients and where both 'MES_MV' and 'MES_GERCON' are not null
unique_patient_group = df_producao[df_producao['PACIENTE'].isin(one_time_patients) & 
                                   df_producao['MES_GERCON'].notnull() & 
                                   df_producao['MES_MV'].notnull() ]

# Displaying the filtered data
unique_patient_group

,MES_GERCON,DIA_GERCON,PACIENTE,GERCON,MES_MV,DIA_MV,ATENDIMENTO_CODIGO,PACIENTE_CODIGO,CONVENIO,MEDICO_CODIGO,...,VALOR,RECURSO,VALOR_RECURSO,TESTE,PEDIDO,ENTREGA,ADAPTAÇÃO,RECURSO_BENGALA,CÓD. PACIENTE,Data da entrega
612,2023-05-01,2023-05-10 10:00:00,ADRIANO GUILHERME SCHMIDT,PRIMEIRA,2023-05-01,2023-05-10 07:50:00,949341.0,1426563.0,SUS - AMBULATORIO,452.0,...,12.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
625,2023-05-01,2023-05-24 09:00:00,NICOLAS MAY DE SOUZA,PRIMEIRA,2023-05-01,2023-05-24 08:55:00,958765.0,1434860.0,SUS - AMBULATORIO,1102.0,...,10.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
635,2023-06-01,2023-06-07 08:30:00,ARTHUR DE PAULA PATRICIO,PRIMEIRA,2023-06-01,2023-06-07 08:24:00,968406.0,1405557.0,SUS - AMBULATORIO,1102.0,...,10.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
726,2023-08-01,2023-08-30 08:00:00,JAQUELINE CARDOSO,PRIMEIRA,2023-08-01,2023-08-30 07:45:00,1020510.0,1447731.0,SUS - AMBULATORIO,1102.0,...,10.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
744,2023-09-01,2023-09-06 08:30:00,REINOLDO HAMMES DA SILVA,FALTANTE,2023-09-01,2023-09-27 09:09:00,1035459.0,1450798.0,SUS - AMBULATORIO,615.0,...,12.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
749,2023-09-01,2023-09-13 08:00:00,LIVIA HINTERHOLZ LINHARES,PRIMEIRA,2023-09-01,2023-09-13 08:17:00,1027938.0,1449259.0,SUS - AMBULATORIO,1102.0,...,10.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
754,2023-09-01,2023-09-27 08:00:00,MARISA BETINS DOS SANTOS,PRIMEIRA,2023-10-01,2023-10-10 11:00:00,1043097.0,1445293.0,SUS - AMBULATORIO,1222.0,...,12.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
770,2023-10-01,2023-10-11 08:00:00,ORCIMAR MENEZES DE QUEIROZ,PRIMEIRA,2023-10-01,2023-10-11 07:51:00,1043608.0,1452389.0,SUS - AMBULATORIO,1282.0,...,12.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
777,2023-10-01,2023-10-11 09:30:00,MATEUS SANTOS DA ROSA,PRIMEIRA,2023-10-01,2023-10-11 08:45:00,1043710.0,1452402.0,SUS - AMBULATORIO,1282.0,...,12.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT


In [61]:
paciente_counts = df_producao['PACIENTE'].value_counts()
one_time_patients = paciente_counts[paciente_counts == 1].index

# Filtering df_producao for these unique patients and where both 'MES_MV' and 'MES_GERCON' are not null
unique_patient_group = df_producao[df_producao['PACIENTE'].isin(one_time_patients) & 
                                   df_producao['MES_GERCON'].notnull() & 
                                   ~df_producao['MES_MV'].notnull()]

# Displaying the filtered data
unique_patient_group.tail(30)

,MES_GERCON,DIA_GERCON,PACIENTE,GERCON,MES_MV,DIA_MV,ATENDIMENTO_CODIGO,PACIENTE_CODIGO,CONVENIO,MEDICO_CODIGO,...,VALOR,RECURSO,VALOR_RECURSO,TESTE,PEDIDO,ENTREGA,ADAPTAÇÃO,RECURSO_BENGALA,CÓD. PACIENTE,Data da entrega
626,2023-06-01,2023-06-07 08:00:00,JOSE LUIZ BRAGAGNOLLO,PRIMEIRA,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
645,2023-06-01,2023-06-21 08:00:00,SERGIO AVELINO ROSA DE SOUZA,PRIMEIRA,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
646,2023-06-01,2023-06-21 08:00:00,IARA MATOS CAOBELLI DA ROSA,FALTANTE,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
647,2023-06-01,2023-06-21 09:00:00,FRANCISCO CARLOS RODRIGUES,PRIMEIRA,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
655,2023-06-01,2023-06-28 08:00:00,JOSE HENRIQUE SEVERO BARBOSA,PRIMEIRA,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
656,2023-06-01,2023-06-28 08:00:00,KAIKE DIOWANE MOREIRA MORAES,PRIMEIRA,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
674,2023-07-01,2023-07-05 08:00:00,JOSE CARLOS CARPES DA SILVA,FALTANTE,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
675,2023-07-01,2023-07-05 08:00:00,LUIZ FELIPE BATISTA DA SILVA,FALTANTE,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
680,2023-07-01,2023-07-05 08:30:00,ALESSANDRO ROGERIO FERNANDES DE OLIVEIRA,PRIMEIRA,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
683,2023-07-01,2023-07-19 08:00:00,TERESINHA DOS SANTOS LIMA,PRIMEIRA,NaT,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT
